# Inverse of the Normal Map

In [1]:
from igmap import *
from gblend.geometry import *

from plotly.subplots import make_subplots
import plotly.graph_objects as go

res = 3
base = "../SphereTriangulation/"
sph_fs = np.loadtxt(base+'triangles%s.txt'%res).astype(int)
sph_vs = np.loadtxt(base+'vertices%s.txt'%res)

print("Spherical points count: %s"%sph_vs.shape[0])

d_min = get_minimal_vertices_distance(sph_vs, sph_fs)
print("Minimal distance between points: %s"%d_min)

fig = go.Figure()
fig.add_trace(go.Mesh3d(x=sph_vs[:,0], y=sph_vs[:,1], z=sph_vs[:,2], i=sph_fs[:,0], j=sph_fs[:,1], k=sph_fs[:,2], color='gray', opacity=1.0, flatshading=True))
fig.add_trace(go.Scatter3d(x=sph_vs[:,0], y=sph_vs[:,1], z=sph_vs[:,2], mode='markers', marker=dict(size=2, color='red')))
fig.update_layout(width=600, height=600)
fig.show()

Spherical points count: 642
Minimal distance between points: 0.3092105761958676


# Implicit surface

In [6]:
# Lets define the surface

#R = 1
#r = .5
#ax = 0
#ay = 0
#f1 = (sqrt(x**2+y**2)- R)**2 + (z+ax*x**2+ay*y**2)**2 - r**2
#f2 = (x-2)**2 + y**2 + z**2 - (.1)**2
#h1 = sigmoid.subs(s,-f1)
#h2 = sigmoid.subs(s,-f2)
#formula = -(h1 + h2 - h1*h2 - 0.5)

b1 = Ball3D()
b1.scale(1/2,1/2,1/3)
b1.translate(-1/2,0,0)

b2 = Ball3D()
b2.scale(1/2,1/2,1/3)
b2.translate(1/2,0,0)

c1 = Cylinder3D()
c1.scale(1/8,1/8,1)
c1.translate(-1/2,0,0)

c2 = Cylinder3D()
c2.scale(1/8,1/8,1)
c2.translate(1/2,0,0)

obj = (b1 | b2)

#obj = Torus3D()
#obj.displace(.1*x**2 - .1*y**2)


formula = 0.5 - obj.f  # 0.5 is the level set we want to extract

#formula = x**2 + y**2 + z**2 - 1
f, fx, fy, fz = compute_jet(formula)

sd = 3
box = [-sd,sd,-sd,sd,-sd,sd]
srf_vs0, srf_fs0 = iso_surface(f, box)
srf_ns0 = get_normals(fx,fy,fz, srf_vs0)

fig = make_subplots(rows=1, cols=2, specs = [[{'type':'scene'}, {'type':'scene'}]], subplot_titles=("Surface", "Normals"))
# set equal aspect ratios for all axes
fig.add_trace(go.Mesh3d(x=srf_vs0[:,0], y=srf_vs0[:,1], z=srf_vs0[:,2], i=srf_fs0[:,0], j=srf_fs0[:,1], k=srf_fs0[:,2], color='blue', opacity=0.25), row=1, col=1)

# Sampling the surface for normals rendering
mesh = tm.Trimesh(srf_vs0, srf_fs0)
points_count = 1000
srf_vs, _ = tm.sample.sample_surface_even(mesh, points_count)

grd_ns, srf_ns = get_normals(fx,fy,fz, srf_vs)


nlx,nly,nlz = get_plotly_normal_lines(srf_vs, srf_ns)
fig.add_trace(go.Scatter3d(x=nlx, y=nly, z=nlz, mode='lines', marker=dict(size=.1, color='green')), row=1, col=1)

# Sampling the surface for inverse of normal map

mesh = tm.Trimesh(srf_vs0, srf_fs0)
points_count = 1000
srf_vs, tmp = tm.sample.sample_surface_even(mesh, points_count)
_,srf_ns = get_normals(fx,fy,fz, srf_vs)

# Computing inverse points and triangles
inv_triangles_points = get_inverse_triangles_points(srf_vs, srf_ns, sph_vs, sph_fs)

for points in inv_triangles_points:
    fig.add_trace(go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], 
                               mode='markers', marker=dict(size=2),showlegend=False),row=1,col=2)


# Visualize results

In [7]:
fig.update_layout(width=1200, height=600)
fig.show()